In [40]:
import pandas as pd
from fuzzywuzzy import process


/Users/user/Documents/GitHub/Thesis_2425/venv/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [89]:
#import mine data
#primary data
mine_pri = pd.read_excel("./Data/new coal/Global-Coal-Mine-Tracker-April-2024 (1).xlsx", sheet_name="Global Coal Mine Tracker (Non-C")
#supplement data
mine_sup1 = pd.read_excel("./Data/new coal/Global-Coal-Mine-Tracker-September-2024-Supplement-v2 (2).xlsx", sheet_name="Historical Production (non-Chin")
mine_sup2 =pd.read_excel("./Data/new coal/Global-Coal-Mine-Tracker-December-2024-Supplement-Historical-Production-from-2018-to-2023.xlsx", sheet_name="Historical Production(non-China")

In [88]:
#import plant data
plant = pd.read_excel("./Data/new coal/Global-Coal-Plant-Tracker-January-2025.xlsx", sheet_name="Units")

#correct the estimated capacity with capacity factor
plant['Est_capacity (MW)']=plant['Capacity (MW)']*plant['Capacity factor']

#import terminal data
#note: the capacity of terminal dataset may be overestimated, because some terminals not only receive 'coal' but 'all cargo'
terminal = pd.read_excel("./Data/new coal/Global-Coal-Terminals-Tracker-December-2024.xlsx", sheet_name="Terminals")

In [90]:
#may further filter by 'status == 'operating'
mine_pri = mine_pri[mine_pri['Country'] == 'India']
plant = plant[plant['Country/Area'] == 'India']
terminal = terminal[terminal['Country/Area'] == 'India']

## Sheet 1: Assets

In [21]:
mine_assets = mine_pri[['GEM Mine ID', 'Mine Name', 'Latitude', 'Longitude', 'Country', 'Production (Mtpa)']]
plant_assets = plant[['GEM location ID', 'Plant name', 'Latitude', 'Longitude','Country/Area', 'Est_capacity (MW)' ]]
terminal_assets = terminal[['GEM Terminal ID', 'Coal Terminal Name', 'Latitude', 'Longitude','Country/Area', 'Capacity (Mt)']]

mine_assets.columns = ['asset_id', 'name', 'latitude', 'longitude', 'country', 'production']
plant_assets.columns = ['asset_id', 'name', 'latitude', 'longitude', 'country', 'production']
terminal_assets.columns = ['asset_id', 'name', 'latitude', 'longitude', 'country', 'production']

mine_assets['product'] = 'coal'
plant_assets['product'] = 'electricity'
terminal_assets['product'] = 'coal'#may be incorrect because some are 'all cargo'

def process_assets(df):
    # 拆分数据：production = '*' 的行 和 其余可计算的行
    star_rows = df[df["production"] == '*']  # 只包含 '*' 生产值的行
    numeric_rows = df[df["production"] != '*'].copy()  # 其余行可进行数值合并

    # 确保 production 是数值类型
    numeric_rows["production"] = pd.to_numeric(numeric_rows["production"], errors="coerce").fillna(0)

    # 仅对数值部分进行 groupby 聚合
    grouped_numeric = numeric_rows.groupby("asset_id", as_index=False).agg({
        "name": "first",
        "latitude": "first",
        "longitude": "first",
        "country": "first",
        "production": "sum",
        "product": "first"
    })

    # 合并 groupby 结果 + production = '*' 的原始行
    return pd.concat([grouped_numeric, star_rows], ignore_index=True)

# 处理各个数据集
mine_assets = process_assets(mine_assets)
plant_assets = process_assets(plant_assets)
terminal_assets = process_assets(terminal_assets)

# 合并所有资产数据
total_assets = pd.concat([mine_assets, plant_assets, terminal_assets], ignore_index=True)

# 输出最终结果
total_assets

/var/folders/wy/k_hfp9ls0msf61dg9qbm3h040000gn/T/ipykernel_7944/1934385193.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mine_assets['product'] = 'coal'
/var/folders/wy/k_hfp9ls0msf61dg9qbm3h040000gn/T/ipykernel_7944/1934385193.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plant_assets['product'] = 'electricity'
/var/folders/wy/k_hfp9ls0msf61dg9qbm3h040000gn/T/ipykernel_7944/1934385193.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

,asset_id,name,latitude,longitude,country,production,product
0,M0464,Adasa Coal Mine,21.347003,78.953545,India,0.61,coal
1,M0465,Adriyala Coal Mine,18.6648,79.579,India,1.54,coal
2,M0466,AKK Coal Mine,23.7905,85.9198,India,4.25,coal
3,M0467,Amadand Coal Mine,23.157942,82.061533,India,0.87,coal
4,M0468,Amalgamated Keshalpur-West Mudidih (AKWMC) Coa...,23.8078,86.3221,India,4.1,coal
...,...,...,...,...,...,...,...
1243,T1415,Visakhapatnam Port Adani Terminal,17.68333,83.2833,India,6.4,coal
1244,T1432,Rewas Port Coal Terminal,18.79178,72.914995,India,66.0,coal
1245,T1383,Adani Hazira Port Coal Terminal,21.08234,72.627231,India,*,coal
1246,T1019,Dahanu Port Terminal,19.97,72.73,India,*,coal


In [22]:
prod_index = total_assets.columns.get_loc('production')

total_assets.insert(prod_index, 'replacement_value', '')

## Sheet 2: Firms

In [23]:
import numpy as np

def process_firms_data(df, owner_column, firm_type):
    """
    通用函数处理 firms 数据

    参数：
    df: DataFrame - 原始数据
    owner_column: str - 所有者列名
    firm_type: str - 企业类型 ('production' 等)

    返回：
    DataFrame - 处理后的数据
    """
    firms = df[[owner_column]].rename(columns={owner_column: 'name'})
    
    # 添加必要的列，初始化为空值
    firms['firm_id'] = np.nan
    firms['final_demand'] = np.nan
    firms['discount_rate'] = np.nan
    firms['unit_price'] = np.nan
    firms['margin'] = np.nan
    firms['leverage_ratio'] = np.nan
    firms['type'] = firm_type  # 设定企业类型
    
    # 调整列顺序
    firms = firms[['firm_id', 'name', 'final_demand', 'discount_rate', 
                   'unit_price', 'margin', 'leverage_ratio', 'type']]
    
    return firms

# 处理三个数据集
mine_firms = process_firms_data(mine_pri, 'Owners', 'production')
plant_firms = process_firms_data(plant, 'Owner', 'production')
terminal_firms = process_firms_data(terminal, 'Owner', 'production')
total_firms = pd.concat([mine_firms, plant_firms, terminal_firms], ignore_index=True)
total_firms.drop_duplicates(subset=['name'], keep='first', inplace=True)

with pd.ExcelWriter('./Data/coal_supply_chain_india.xlsx') as writer:
    total_assets.to_excel(writer, sheet_name='assets', index=False)
    total_firms.to_excel(writer, sheet_name='firms', index=False)

In [24]:
total_firms

,firm_id,name,final_demand,discount_rate,unit_price,margin,leverage_ratio,type
0,NaN,Western Coalfields Ltd,NaN,NaN,NaN,NaN,NaN,production
1,NaN,The Singareni Collieries Co Ltd,NaN,NaN,NaN,NaN,NaN,production
2,NaN,Central Coalfields Ltd,NaN,NaN,NaN,NaN,NaN,production
3,NaN,South Eastern Coalfields Ltd,NaN,NaN,NaN,NaN,NaN,production
5,NaN,Bharat Coking Coal Ltd,NaN,NaN,NaN,NaN,NaN,production
...,...,...,...,...,...,...,...,...
2548,NaN,Shree Cement,NaN,NaN,NaN,NaN,NaN,production
2549,NaN,Tata Group,NaN,NaN,NaN,NaN,NaN,production
2552,NaN,V.O. Chidambaranar Port Trust,NaN,NaN,NaN,NaN,NaN,production
2555,NaN,Visakhapatnam Port Trust (VPT),NaN,NaN,NaN,NaN,NaN,production


## Sheet 4: Supply Chain

In [26]:
# 1️⃣ 创建 supply_chain DataFrame
supply_chain = pd.DataFrame(columns=['supplier', 'client', 'recipe', 'product'])
supply_chain['supplier'] = terminal['GEM Terminal ID'].unique()

# 3️⃣ 筛选 terminal 里 'Coal Source' 包含不同国家的 'GEM Terminal ID'
indonesia_terminal_ids = terminal[terminal['Coal Source'].str.contains('Indonesia', case=False, na=False, regex=True)]['GEM Terminal ID'].unique()
australia_terminal_ids = terminal[terminal['Coal Source'].str.contains('Australia', case=False, na=False, regex=True)]['GEM Terminal ID'].unique()
mozambique_terminal_ids = terminal[terminal['Coal Source'].str.contains('Mozambique', case=False, na=False, regex=True)]['GEM Terminal ID'].unique()
south_africa_terminal_ids = terminal[terminal['Coal Source'].str.contains('South Africa', case=False, na=False, regex=True)]['GEM Terminal ID'].unique()

# 4️⃣ 筛选 plant 里 'Coal Source' 包含不同国家的 'GEM Plant ID'
indonesia_plant_ids = plant[plant['Coal source'].str.contains('Indonesia', case=False, na=False, regex=True)]['GEM location ID'].unique()
australia_plant_ids = plant[plant['Coal source'].str.contains('Australia', case=False, na=False, regex=True)]['GEM location ID'].unique()
mozambique_plant_ids = plant[plant['Coal source'].str.contains('Mozambique', case=False, na=False, regex=True)]['GEM location ID'].unique()
south_africa_plant_ids = plant[plant['Coal source'].str.contains('South Africa', case=False, na=False, regex=True)]['GEM location ID'].unique()

# 5️⃣ 创建新的 DataFrame 存储扩展的匹配关系
expanded_rows = []

# 6️⃣ 遍历符合条件的 terminal ID，并为其复制多行，使其能匹配多个 plant ID
for terminal_id in indonesia_terminal_ids:
    for plant_id in indonesia_plant_ids:
        expanded_rows.append({'supplier': terminal_id, 'client': plant_id, 'recipe': '', 'product': ''})

for terminal_id in australia_terminal_ids:
    for plant_id in australia_plant_ids:
        expanded_rows.append({'supplier': terminal_id, 'client': plant_id, 'recipe': '', 'product': ''})

for terminal_id in mozambique_terminal_ids:
    for plant_id in mozambique_plant_ids:
        expanded_rows.append({'supplier': terminal_id, 'client': plant_id, 'recipe': '', 'product': ''})

for terminal_id in south_africa_terminal_ids:
    for plant_id in south_africa_plant_ids:
        expanded_rows.append({'supplier': terminal_id, 'client': plant_id, 'recipe': '', 'product': ''})

# 5️⃣ 转换为 DataFrame 并追加到 supply_chain
expanded_df = pd.DataFrame(expanded_rows)

# 6️⃣ 将扩展后的数据追加到 supply_chain
supply_chain = pd.concat([supply_chain, expanded_df], ignore_index=True)
supply_chain


,supplier,client,recipe,product
0,T1383,NaN,NaN,NaN
1,T1180,NaN,NaN,NaN
2,T1183,NaN,NaN,NaN
3,T1022,NaN,NaN,NaN
4,T1019,NaN,NaN,NaN
...,...,...,...,...
286,T1170,L100000102290,,
287,T1166,L100000102556,,
288,T1166,L100000102290,,
289,T1170,L100000102047,,


In [27]:
# 创建一个新的 DataFrame 保存 plant 数据，并删除 'Coal Source' 中包含特定国家的行
plant_filtered = plant[
    ~plant['Coal source'].str.contains('Indonesia|Australia|South Africa|Mozambique', case=False, na=False, regex=True)
].copy()
plant_filtered = plant_filtered[
    plant['Coal source'].str.contains('mported', case=False, na=False, regex=True)
].copy()
# 1️⃣ 找出 supply_chain 里没有 client 的 supplier（GEM Terminal ID）
suppliers_without_clients = supply_chain[supply_chain['client'].isna()]['supplier'].unique()

# 2️⃣ 获取 plant_filtered 中的所有 GEM Plant ID
filtered_plant_ids = plant_filtered['GEM location ID'].unique()

# 3️⃣ 仅针对没有 client 的 supplier，复制行并填充所有符合条件的 GEM Plant ID
expanded_rows_imported = []
for supplier in suppliers_without_clients:
    for plant_id in filtered_plant_ids:  # 仅给没有 client 的 supplier 赋予所有 plant_filtered 里的 GEM Plant ID
        expanded_rows_imported.append({
            'supplier': supplier,
            'client': plant_id,
            'recipe': '',
            'product': ''
        })

# 4️⃣ 转换为 DataFrame 并追加到 supply_chain
expanded_df_imported = pd.DataFrame(expanded_rows_imported)
supply_chain = pd.concat([supply_chain, expanded_df_imported], ignore_index=True)
supply_chain = supply_chain.dropna(subset=['client']).reset_index(drop=True)



/var/folders/wy/k_hfp9ls0msf61dg9qbm3h040000gn/T/ipykernel_1116/1893165705.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  plant_filtered = plant_filtered[


In [ ]:
# 1️⃣ 在 supply_chain 数据集中创建四个新列
supply_chain['supplier_lat'] = None
supply_chain['supplier_lon'] = None
supply_chain['client_lat'] = None
supply_chain['client_lon'] = None

# 2️⃣ 通过 GEM Terminal ID 从 terminal 数据集中查找 supplier 的 lat/lon（去重）
supplier_coords = terminal[['GEM Terminal ID', 'Latitude', 'Longitude']].drop_duplicates(subset=['GEM Terminal ID']).set_index('GEM Terminal ID')

# 3️⃣ 进行匹配并展开成两列
supplier_mapped = supply_chain['supplier'].map(supplier_coords.to_dict(orient='index'))
supplier_mapped = supplier_mapped.apply(lambda x: pd.Series([x['Latitude'], x['Longitude']]) if isinstance(x, dict) else pd.Series([None, None]))

# 4️⃣ 赋值到 supply_chain
supply_chain[['supplier_lat', 'supplier_lon']] = supplier_mapped

# 5️⃣ 通过 GEM Location ID 从 plant 数据集中查找 client 的 lat/lon（去重）
client_coords = plant[['GEM location ID', 'Latitude', 'Longitude']].drop_duplicates(subset=['GEM location ID']).set_index('GEM location ID')

# 6️⃣ 进行匹配并展开成两列
client_mapped = supply_chain['client'].map(client_coords.to_dict(orient='index'))
client_mapped = client_mapped.apply(lambda x: pd.Series([x['Latitude'], x['Longitude']]) if isinstance(x, dict) else pd.Series([None, None]))

# 7️⃣ 赋值到 supply_chain
supply_chain[['client_lat', 'client_lon']] = client_mapped

import numpy as np

# 计算两个经纬度点之间的距离（Haversine 公式）
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # 地球半径（单位：公里）
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# 计算 supply_chain 数据集中每一对 supplier-client 之间的距离
supply_chain['distance_km'] = supply_chain.apply(
    lambda row: haversine(row['supplier_lat'], row['supplier_lon'], row['client_lat'], row['client_lon'])
    if not any(pd.isna([row['supplier_lat'], row['supplier_lon'], row['client_lat'], row['client_lon']])) else np.nan,
    axis=1
)

# 仅保留距离小于 1000 公里的行
supply_chain = supply_chain[supply_chain['distance_km'] < 1000].reset_index(drop=True)

# 通过 client ID (GEM location ID) 在 plant 数据集中找到对应的 Coal Type
client_coal_type = plant[['GEM location ID', 'Coal type']].drop_duplicates(subset=['GEM location ID']).set_index('GEM location ID')

# 进行匹配，将 Coal Type 填充到 supply_chain 的 'product' 列
supply_chain['product'] = supply_chain['client'].map(client_coal_type['Coal type'])
supply_chain = supply_chain.drop(columns=['supplier_lat', 'supplier_lon', 'client_lat', 'client_lon', 'distance_km'], errors='ignore')

从这里开始处理plant和mine之间的联系

In [93]:
# 删除 plant 数据集中 'Coal source' 为空的行
plant = plant.dropna(subset=['Coal source']).copy()
plant = plant[
    (plant['Coal source'].str.lower() != 'imported') &
    (plant['Coal source'] != 'Imported coal') &
    (plant['Coal source'] != 'Imported from Indonesia') &
    (plant['Coal source'] != 'Imported (Indonesia)') &
    (plant['Coal source'] != 'Imported Indonesian and Australian coal')
]
mine_pri = mine_pri.drop_duplicates(subset=['GEM Mine ID'], keep='first')


In [ ]:
import time
import openai

# 设置 OpenAI API Key
client = openai.OpenAI(api_key='你的api')

# 1️⃣ 创建 supply_chain_mine DataFrame
supply_chain_mine = pd.DataFrame(columns=['supplier', 'client', 'recipe', 'product'])
expanded_rows = []

# 2️⃣ 遍历 `mine_pri` 进行匹配
for _, mine_row in mine_pri.iterrows():
    mine_name = mine_row['Mine Name']
    owner_name = mine_row['Owners']

    for _, plant_row in plant.iterrows():
        coal_source = plant_row['Coal source']

        print(f"Processing: {mine_name} {owner_name} → {coal_source}")

        # 3️⃣ 生成 OpenAI API 请求的 prompt
        prompt = f"""
        You are an expert in the Indian coal industry. Your task is to determine whether a given coal source originates from a specific Indian mine or is supplied by a specific Indian owner.
        
        **Given Data:**
        - **Coal Source (from power plant data):** "{coal_source}"
        - **Mine Name (from mine data):** "{mine_name}"
        - **Owner (from mine data):** "{owner_name}"

        **Instructions:**
        - Consider whether the **Coal Source** refers to the **Mine Name** directly or belongs to the **Owner**.
        - If the **Coal Source** clearly refers to the **Mine Name** or **Owner**, return `'Yes'`.
        - If there is no clear relationship, return `'No'`.

        Answer strictly with 'Yes' or 'No' and nothing else.
        """

        # 4️⃣ 调用 OpenAI API，使用 `gpt-3.5-turbo`（更快更便宜）
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            timeout=30  # 预防 API 超时
        )

        # 5️⃣ 解析 API 响应
        match_result = response.choices[0].message.content.strip().lower()
        print(f"🔍 GPT-3.5 结果: {match_result}")

        # 6️⃣ 如果匹配成功，则存储数据
        if match_result == "yes":
            expanded_rows.append({
                'supplier': mine_row['GEM Mine ID'],
                'client': plant_row['GEM location ID'],
                'recipe': '',
                'product': plant_row['Coal type']
            })

        # 7️⃣ **加入时间间隔，防止 API 速率限制**
        time.sleep(0.3)  # 0.3秒延迟，防止 API 过载

# 8️⃣ 转换为 DataFrame 并追加到 `supply_chain_mine`
supply_chain_mine = pd.concat([supply_chain_mine, pd.DataFrame(expanded_rows)], ignore_index=True)

Processing: Adasa Coal Mine Western Coalfields Ltd → domestic
🔍 GPT-3.5 结果: yes
Processing: Adasa Coal Mine Western Coalfields Ltd → domestic
🔍 GPT-3.5 结果: yes
Processing: Adasa Coal Mine Western Coalfields Ltd → South Eastern Coalfields
🔍 GPT-3.5 结果: no
Processing: Adasa Coal Mine Western Coalfields Ltd → South Eastern Coalfields
🔍 GPT-3.5 结果: no
Processing: Adasa Coal Mine Western Coalfields Ltd → South Eastern Coalfields
🔍 GPT-3.5 结果: no
Processing: Adasa Coal Mine Western Coalfields Ltd → South Eastern Coalfields
🔍 GPT-3.5 结果: no
Processing: Adasa Coal Mine Western Coalfields Ltd → South Eastern Coalfields
🔍 GPT-3.5 结果: no
Processing: Adasa Coal Mine Western Coalfields Ltd → MCL mines and Mandakini Block in Talcher
🔍 GPT-3.5 结果: no
Processing: Adasa Coal Mine Western Coalfields Ltd → MCL mines and Mandakini Block in Talcher
🔍 GPT-3.5 结果: no
Processing: Adasa Coal Mine Western Coalfields Ltd → Khadia coal mine extension
🔍 GPT-3.5 结果: no
Processing: Adasa Coal Mine Western Coalfields

KeyboardInterrupt: 

In [ ]:
max_requests = 5  # 先测试 5 个请求，看看 API 是否能稳定运行
counter = 0

for _, mine_row in mine_pri.iterrows():
    for _, plant_row in plant.iterrows():
        if counter >= max_requests:
            break
        counter += 1

        print(f"Processing #{counter}: {mine_row['Mine Name']} → {plant_row['Coal source']}")

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            timeout=30
        )

        print(response.choices[0].message.content.strip())


Processing #1: Adasa Coal Mine → imported
No
Processing #2: Adasa Coal Mine → imported
No
Processing #3: Adasa Coal Mine → domestic
No
Processing #4: Adasa Coal Mine → domestic
No
Processing #5: Adasa Coal Mine → South Eastern Coalfields
No
